In [ ]:

#     В ЭТОМ КОДЕ:

#     + с помощью генератора ссылок на посты в группе ССММ кластеризуем посты (про выборы/нет, анонсы/результаты выборов)
#     + посты с результатами выборов конвертируем в словарики со всей информацией об итогах выборов
#   !!! посты с анонсами выборов конвертируем в словарики (имя, курс-поток-группа, текст программы)
#   !!! последние два пункта адаптировать для постов про магистров
#   !!! расставляем флаги предпочтений кандидатов, исходя из текста программ


In [285]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

In [447]:
# СЛОВАРЬ ИНФОРМАЦИИ ИЗ ПОСТА

def vk_post_info_by_link(link_vk):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # ПРОВЕРЯЮ, СУЩЕСТВУЕТ ЛИ ПОСТ
    #     <div class="service_msg service_msg_error">Запись не найдена.</div>
    service_msg_error = soup.find('div', attrs={'class': 'service_msg service_msg_error'})
    
    # ПРОВЕРЯЮ, ЭТО ПОСТ ИЛИ КОММЕНТ    
    # когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
    # когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />
    rel_canonical = soup.find('link', attrs={'rel': 'canonical'})
#     rel_canonical_list = set(re.findall('_[\d]+', str(rel_canonical)))
    rel_canonical_reply = re.findall('reply=', str(rel_canonical))
            
    # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
    # <div class="wi_body">
    #     <div class="pi_text">
    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    if wi_body:
        pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
        text_lines = [line for line in pi_text.stripped_strings]
        # определить тип поста
        # анонсы: 'Орфография и пунктуация сохранены.'
        # результаты: 'Таким образом, выборы признаются состоявшимися'
        announcement_list = ['announcement' for line in pi_text.stripped_strings if line.find('Орфография и пунктуация сохранены.') > 0]
        result_list = ['result' for line in pi_text.stripped_strings if line.find('Таким образом, выборы признаются состоявшимися.') > 0]
    else:
        text_lines = []
        announcement_list = []
        result_list = []
    
    if service_msg_error:
        link_type = 'page not found'
    elif rel_canonical_reply:
        link_type = 'comment'
    elif ('#выборы@stud_mm' in text_lines) or ('#выборы_ссмм' in text_lines):
        link_type = 'election'
    else:
        link_type = ''
    
    if len(announcement_list) > 0:
        link_subtype = 'announcement'
    elif len(result_list) > 0:
        link_subtype = 'result'
    else:
        link_subtype = 'other'
        
    # ВЫТАСКИВАЮ ДАТУ
    # <div class="wi_info">
    #     <span class="wi_date">
    months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    wi_info = soup.find('div', attrs={'class': 'wi_info'})
    if wi_info:
        wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
        date_list = [date for date in wi_date.stripped_strings]
        link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
#         month = re.findall(r'[\w]{3}', date_list[0])[0]
        link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
        link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
        link_date = date(link_year_no, link_month_no, link_day_no)
        link_date_time = date_list[0]
        link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
    else:
        link_year_no = 0
#         month = ''
        link_month_no = 0
        link_day_no = 0
        link_date = date(1970, 1, 1)
        link_date_time = ''
        link_time = ''

    dict_link = {}
    dict_link[link_vk] = {'link_type': link_type, 'link_subtype': link_subtype, 'link_date': link_date, 'link_year_no': link_year_no, 'link_month_no': link_month_no, 'link_day_no': link_day_no, 'link_date_time': link_date_time, 'link_time': link_time, 'list_of_lines': text_lines}
    
    return dict_link


In [ ]:

# выскивает из строки номера курса и потока (в предположении, что они в этой строке есть)
def year_stream_no_by_line(line):
    year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line))[0]
    year_stream_line_d = re.sub('\D', '', year_stream_line)
    year_no = int(year_stream_line_d[0])
    stream_no = int(year_stream_line_d[1])
    year_stream_no = str(year_no) + '.' + str(stream_no)
    return year_stream_no, year_no, stream_no


In [529]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО КАНДИДАТОВ (по списку строк с текстом)
# ПОКА БЕЗ МАГИСТРАТУРЫ

# по списку строк с текстом составляем СЛОВАРЬ: имя: курс-поток, группа, номер первой строки в тексте
def nominated_by_list(list_of_lines):
    nominated_dict = {}
    year_stream_no, year_no, stream_no = 0, 0, 0
    
    for i in range(len(list_of_lines)):
        
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
            
        if len(re.findall(r'\s[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i])) > 0:
            name_line_no = i
            name_line = (re.findall(r'\s[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i]))[0]
            name_list = re.split(', ',name_line)
            name = (re.sub('\W\s', '', name_list[0])).strip().title()
            academic_group_no = int(re.sub('\D', '', name_list[1]))
            another_one_dict = {}
            another_one_dict[name] = {
                'year_stream_no': year_stream_no, 
                'year_no': year_no, 
                'stream_no': stream_no, 
                'academic_group_no': academic_group_no, 
                'name_line_no': name_line_no
            }
            nominated_dict = {**nominated_dict, **another_one_dict}
            
    # первый и последний индексы программы кандидата в списке
    name_list = list(nominated_dict.keys())
    start_index_list = [nominated_dict[key]['name_line_no'] for key in list(nominated_dict.keys())]
    last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
    last_index_list.append(len(list_of_lines)-1)
    
    dict_of_nominated_upd = {}
    
    for name in name_list:
        dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
    # обновить словарь: имя: номер группы, текст программы
    for name in dict_of_nominated_upd:
        program_lines = []

        for k in range(dict_of_nominated_upd[name][0]+1, dict_of_nominated_upd[name][1]+1):
            program_lines.append(list_of_lines[k])
        program_txt = '\n'.join(program_lines)

        nominated_dict[name].update({'program_txt': program_txt})
        del nominated_dict[name]['name_line_no']

    return nominated_dict


In [530]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С АНОНСАМИ ВЫБОРОВ

announcement_links = {}

for num in range(2200, 2250):
    test_link = 'https://vk.com/wall-33327655_' + str(num)
    test_dict = vk_post_info_by_link(test_link)
    for d in test_dict.values():
        if d['link_type'] == 'election' and d['link_subtype'] == 'announcement':
            announcement_links = {**announcement_links, **test_dict}
            
for link in announcement_links:
    announcement_links[link].update({'nominated_info': nominated_by_list(announcement_links[link]['list_of_lines'])})
    del announcement_links[link]['list_of_lines']

announcement_links

{'https://vk.com/wall-33327655_2202': {'link_type': 'election',
  'link_subtype': 'announcement',
  'link_date': datetime.date(2014, 11, 13),
  'link_year_no': 2014,
  'link_month_no': 11,
  'link_day_no': 13,
  'link_date_time': '13 ноя 2014 в 21:39',
  'link_time': '21:39',
  'nominated_info': {'Суркова Анастасия Юрьевна': {'year_stream_no': '2.2',
    'year_no': 2,
    'stream_no': 2,
    'academic_group_no': 209,
    'program_txt': '«Подавая заявку в прошлом году в студсовет, являлась наивной непонимающей девочкой, не представлявшей, что же есть эта студенческая общественная деятельность. Теперь же с полным осознанием дела пишу этот текст повторно, что значит, что мои желания оправдались и выбор хоть чуть-чуть, но осмыслен :)\nИтак, почему же надо выбрать меня?\nВо-первых, я, на самом деле, являюсь инициативным человеком. Пока пыл во мне не выветрится, я могу создавать мероприятия, агитировать на участие людей, делать что-то весёлое и зажигательное, а главное не оправдывать бездейс

In [514]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО РЕЗУЛЬТАТЫ ВЫБОРОВ (по списку строк с текстом)


# по списку строк с кандидатами составляем СЛОВАРЬ: имя: голоса, статус
def stream_election_result_by_list(nominated_with_votes):
    dict_nominated = {}

    for line in nominated_with_votes:
        name = ((re.findall(r'[\w\s]+', line))[0]).strip().title()
        votes_line = (re.findall(r'\s[\d]+\sголос', line))[0]
        votes_cnt = int(re.sub('\D', '', votes_line))
        dict_nominated[name] = {'votes_cnt': votes_cnt}
        if (re.findall(r' избран', line)) and not (re.findall(r' не избран', line)):
            elected_flg = 1
        else:
            elected_flg = 0
        dict_nominated[name].update({'elected_flg': elected_flg})
    return dict_nominated


# вся инфа про результаты выборов на потоке
# ПОКА БЕЗ МАГИСТРАТУРЫ
def stream_election_info_by_dict(list_of_lines):
    nominated_with_votes = []
    year_stream_no = ''
    year_no, stream_no = 0, 0
    voted_cnt, stream_size, invalid_cnt, NOTA_cnt = 0, 0, 0, 0
    
    for i in range(len(list_of_lines)):
        
#         if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
#             year_stream_line_no = i
#             year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]))[0]
#             year_stream_line_d = re.sub('\D', '', year_stream_line)
#             year_no = int(year_stream_line_d[0])
#             stream_no = int(year_stream_line_d[1])
#             year_stream_no = str(year_no) + '.' + str(stream_no)
            
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
            
        if re.findall('Явка: ', list_of_lines[i]):
#             stat_line_no = i
            turnout_line = (re.findall(r'[\d]+\s[\w]+\sиз\s[\d]+', list_of_lines[i]))[0]
            voted_cnt = int(((re.findall(r'[\d]+\s', turnout_line))[0]).strip())
            stream_size = int(((re.findall(r'\s[\d]+', turnout_line))[0]).strip())
            
        if len(re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i])) == 0: # отдельно, т.к. может такой записи не быть
            pass
        else:
            invalid_line_no = i
            invalid_line = (re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i]))[0]
            invalid_cnt = re.sub('\D', '', invalid_line)
            
        if re.findall('Против всех', list_of_lines[i]):
            NOTA_line_no = i
            NOTA_line = (re.findall(r'\s[\d]+\sголос', list_of_lines[i]))[0]
            NOTA_cnt = re.sub('\D', '', NOTA_line)
            
        if re.findall('[\w]+\:\s[\d]+\sголос', list_of_lines[i]) and not (re.findall('Против всех', list_of_lines[i])):
            nominated_with_votes.append(list_of_lines[i])
            
        if re.findall('Подтверждено ответственн', list_of_lines[i]):
            organizer_line = (re.findall(r'за\sвыборы\s[\w\s,]+', list_of_lines[i]))[0]
            organizer_line = organizer_line[10::]
            organizer_list = re.split(', | и ',organizer_line)
    
    dict_nominated_with_votes = stream_election_result_by_list(nominated_with_votes)
    
#     dict_stream_elections = {}
#     dict_stream_elections[year_stream_no] = {
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'stream_size': stream_size, 
        'voted_cnt': voted_cnt,
        'invalid_cnt': invalid_cnt,
        'NOTA_cnt': NOTA_cnt,
        'organizers': organizer_list,
        'dict_nominated_with_votes': dict_nominated_with_votes
    }
    
    return dict_stream_elections


In [515]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ

result_links = {}

for num in range(2200, 2300):
    test_link = 'https://vk.com/wall-33327655_' + str(num)
    test_dict = vk_post_info_by_link(test_link)
    for d in test_dict.values():
        if d['link_type'] == 'election' and d['link_subtype'] == 'result':
            result_links = {**result_links, **test_dict}
            
for link in result_links:
    result_links[link].update({'stream_election_info': stream_election_info_by_dict(result_links[link]['list_of_lines'])})
    del result_links[link]['list_of_lines']

result_links

{'https://vk.com/wall-33327655_2215': {'link_type': 'election',
  'link_subtype': 'result',
  'link_date': datetime.date(2014, 11, 17),
  'link_year_no': 2014,
  'link_month_no': 11,
  'link_day_no': 17,
  'link_date_time': '17 ноя 2014 в 21:06',
  'link_time': '21:06',
  'stream_election_info': {'year_stream_no': '',
   'year_no': 0,
   'stream_no': 0,
   'stream_size': 135,
   'voted_cnt': 87,
   'invalid_cnt': 0,
   'NOTA_cnt': '7',
   'organizers': ['Глунчадзе Ираклием'],
   'dict_nominated_with_votes': {'Суркова Анастасия Юрьевна': {'votes_cnt': 53,
     'elected_flg': 1},
    'Денисов Илья Олегович': {'votes_cnt': 61, 'elected_flg': 1},
    'Тюрина Татьяна Игоревна': {'votes_cnt': 48, 'elected_flg': 1}}}},
 'https://vk.com/wall-33327655_2216': {'link_type': 'election',
  'link_subtype': 'result',
  'link_date': datetime.date(2014, 11, 17),
  'link_year_no': 2014,
  'link_month_no': 11,
  'link_day_no': 17,
  'link_date_time': '17 ноя 2014 в 21:21',
  'link_time': '21:21',
  'stre

In [ ]:


# ДАЛЕЕ НЕАКТУАЛЬНО



In [531]:
# образец:

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2225')
# link_info

# for key in link_info:
#     list_of_lines = link_info[key]['list_of_lines']
# list_of_lines

In [532]:
# nominated_dict = nominated_by_list(list_of_lines)
# nominated_dict

In [455]:
# примеры:
    
# 33327655_2225 - announcemen
# 33327655_2252 - result
# 33327655_8800 - коммент

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2252')
# link_info

In [456]:
# в интервале ссылок составили словарь для всех постов про выборы

# election_links = {}

# for num in range(2200, 2250):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if d['link_type'] == 'election':
#             election_links = {**election_links, **test_dict}
        
# election_links

In [457]:
# not_found_links = {}

# for num in range(2200, 2230):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if d['link_type'] == 'page not found':
#             not_found_links = {**not_found_links, **test_dict}
        
# not_found_links

In [458]:
# other_links = {}

# for num in range(2200, 2230):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_dict = vk_post_info_by_link(test_link)
#     for d in test_dict.values():
#         if not (d['link_type'] in ('election', 'page not found')):
#             other_links = {**other_links, **test_dict}
        
# other_links

In [450]:
# работа с постом с РЕЗУЛЬТАТАМИ выборов

# link_info = vk_post_info_by_link('https://vk.com/wall-33327655_2252')
# link_info

In [451]:
# пока некорректное присваивание в цикле, но для примера с 1 ссылкой сойдет

# for d in link_info.values():
#     list_of_lines = d['list_of_lines']
# list_of_lines

In [155]:
# функция, которая по адресу ссылки возвращает список строк текста

# <div class="wi_body">
#     <div class="pi_text">

def vk_post_lines_by_link(link_vk):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    # ВЫТАЩИЛ НУЖНЫЙ БЛОК ТЕКСТА ПОСТРОЧНО
    
    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    if wi_body:
        pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
        text_lines = [line for line in pi_text.stripped_strings]
    else:
        text_lines = []
        
    return text_lines

In [288]:
link_lines_2 = vk_post_lines_by_link_2('https://vk.com/wall-33327655_2225')
# 33327655_2225 - announcemen
# 33327655_2252 - result
link_lines_2


{'https://vk.com/wall-33327655_2225': {'type': 'announcement',
  'list of lines': ['#выборы@stud_mm',
   'Насыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.',
   'Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».',
   '1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)',
   '— НАГОРНЫХ Яна Валерьевна, группа 108',
   '«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и яркая внеучебная жизнь, вот и мне хочется участвовать в организации т

In [134]:
# функция, которая по адресу ссылки возвращает дату поста (или год?)
# <div class="wi_info">
#     <span class="wi_date">

def vk_post_date_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    wi_info = soup.find('div', attrs={'class': 'wi_info'})
    if wi_info:
        wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
        date_list = [date for date in wi_date.stripped_strings]
        return date_list[0]
    else:
        return ''

def vk_post_year_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    wi_info = soup.find('div', attrs={'class': 'wi_info'})
    if wi_info:
        wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
        date_list = [date for date in wi_date.stripped_strings]
        return re.findall(r'[\d]{4}', date_list[0])[0]
    else:
        return ''


In [135]:
link_date = vk_post_date_by_link('https://vk.com/wall-33327655_2220')
print(link_date)
link_year = vk_post_year_by_link('https://vk.com/wall-33327655_2220')
print(link_year)

18 ноя 2014 в 23:10
2014


In [88]:
# функция, которая по адресу ссылки возвращает список строк текста

# <div class="wi_body">
#     <div class="pi_text">

def vk_post_lines_by_link(link_vk_announcement):
    req_announcement = requests.get(link_vk_announcement)
    html_doc = req_announcement.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    # ВЫТАЩИЛ НУЖНЫЙ БЛОК ТЕКСТА ПОСТРОЧНО
    
    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    if wi_body:
        pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
        return [line for line in pi_text.stripped_strings] 
    else:
        return []
    

#     вариант на случай, если блоков текста несколько, но у нас он один (если не смотреть в комменты)
#     text_by_lines = []
#     for wi_body in soup.find_all('div', attrs={'class': 'wi_body'}):
#         pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
#         text_by_strings = [line for line in pi_text.stripped_strings]
#         text_by_lines.append(text_by_strings)
#     if len(text_by_lines) > 0:
#         return text_by_lines[0]
#     else:
#         return []

In [459]:
# list_of_lines = vk_post_lines_by_link('https://vk.com/wall-33327655_2220')
# print(list_of_lines)

In [12]:
# сформированный руками список кандидатов из этого текста

list_of_nominated = [
    'НЕЧАЕВ Никита Сергеевич',
    'МИЗРАКИ Вячеслав Алексеевич',
    'ЯШИН Ростислав Игоревич',
    'ЗМИЕВСКИЙ Леонид Вячеславович',
    'ТАИБОВ Али Зайнудин Гусейнович',
    'КАРЯКИН Игорь Андреевич'
]

In [50]:
# словарик для индексов строк, соответствующих началу программы кандидата

dict_of_nominated = {}

for line in list_of_lines:
    for name in list_of_nominated:
        if line.find(name) > 0:
            dict_of_nominated[name] = list_of_lines.index(line)
#             print(line,name)
# print(dict_of_nominated)

# на всякий случай: отсортировать словарь по ключу
dict(sorted(dict_of_nominated.items(), key=lambda item: item[1]))


{'НЕЧАЕВ Никита Сергеевич': 4,
 'МИЗРАКИ Вячеслав Алексеевич': 11,
 'ЯШИН Ростислав Игоревич': 13,
 'ЗМИЕВСКИЙ Леонид Вячеславович': 15,
 'ТАИБОВ Али Зайнудин Гусейнович': 17,
 'КАРЯКИН Игорь Андреевич': 23}

In [51]:
# первый и последний индексы программы кандидата в списке

name_list = list(dict_of_nominated.keys())
# print(name_list)

start_index_list = list(dict_of_nominated.values())
# print(start_index_list)

last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
last_index_list.append(len(list_of_lines)-1)
# print(last_index_list)

dict_of_nominated_upd = {}

for name in dict_of_nominated:
    dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
print(dict_of_nominated_upd)

{'НЕЧАЕВ Никита Сергеевич': [4, 10], 'МИЗРАКИ Вячеслав Алексеевич': [11, 12], 'ЯШИН Ростислав Игоревич': [13, 14], 'ЗМИЕВСКИЙ Леонид Вячеславович': [15, 16], 'ТАИБОВ Али Зайнудин Гусейнович': [17, 22], 'КАРЯКИН Игорь Андреевич': [23, 34]}


In [49]:
# новый словарь: имя: номер группы, текст программы


dict_of_nominated_full = {}

for name in dict_of_nominated_upd:
    original_name = list_of_lines[dict_of_nominated_upd[name][0]]
    tail_name = re.sub(name, '', original_name)
    academic_group_no = re.sub('\D', '', tail_name)

    program_lines = []
    for k in range(dict_of_nominated_upd[name][0]+1,dict_of_nominated_upd[name][1]+1):
        program_lines.append(list_of_lines[k])
    program_txt = '\n'.join(program_lines)
    
#     print(name+':')
#     print(academic_group_no)
#     print(program_txt)
#     print('\n')
    
    dict_of_nominated_full[name] = [academic_group_no, program_txt]

dict_of_nominated_full

{'НЕЧАЕВ Никита Сергеевич': ['102',
  '«Я постоянно что-то пытаюсь организовать в кругу своих знакомых, нередко получается весьма хорошо (например, летом было 3 встречи абитуриентов МГУ), поэтому, если попаду в Студсовет, то моими главными задачами будут: во-первых, представление интересов первого курса МехМата МГУ в тех рамках, которые мне будут доступны и, во-вторых, организация небольших ивентов на МехМате (есть уже идея с вечером настольных игр в ГЗ в какую-нибудь пятницу и еще пара пока что сырых проектов)\nКак-то так:)\nО себе?\nvk.com/nnsf0sker\n17 лет, русский, учился на дому, снимался в кино, играл на барабанах и много разных странностей творил :)\nP. S ВКонтакте имя измененно чисто случайно, но Валентин - это Никита)»'],
 'МИЗРАКИ Вячеслав Алексеевич': ['101',
  '«вступить в рабочую группу Праздники»'],
 'ЯШИН Ростислав Игоревич': ['104',
  '«Хочу быть полезным своему университету.»'],
 'ЗМИЕВСКИЙ Леонид Вячеславович': ['103',
  '«Студенческий совет занимается очень важной ра

In [60]:
# извлекаем номера курса и потока

year_stream_line = list_of_lines[min(start_index_list)-1]
year_stream_line = re.sub('\D', '', year_stream_line)
year_no = year_stream_line[0]
stream_no = year_stream_line[1]
year_stream_no = year_no + '.' + stream_no

year_stream_no

'1.1'

In [65]:
# 33327655_2225 - пост
# 33327655_2229 - коммент
# 33327655_15000 - не существует

link_testt = 'https://vk.com/wall-33327655_15000'
req_testt = requests.get(link_testt)

req_testt.status_code

200

In [ ]:
# список ссылок на посты про выборы

In [69]:
test_link = 'https://vk.com/wall-33327655_2225'

test_list = vk_post_lines_by_link(test_link)
if test_list[0] == '#выборы@stud_mm':
    print('Yes')

Yes


In [289]:
# election_links = []

# for num in range(2200, 2300):
#     test_link = 'https://vk.com/wall-33327655_' + str(num)
#     test_list = vk_post_lines_by_link(test_link)
    
#     if test_list == []:
#         pass
#     elif '#выборы@stud_mm' in test_list:
#         election_links.append(test_link)
        
# election_links

In [290]:
# то же самое, но мб побыстрее

# election_links_2 = ['https://vk.com/wall-33327655_' + str(num) for num in range(2200, 2300) if '#выборы@stud_mm' in vk_post_lines_by_link('https://vk.com/wall-33327655_' + str(num))]
# election_links_2


In [291]:
# год по списку ссылок

# election_years_2 = [vk_post_year_by_link(link) for link in election_links_2]
# election_years_2

In [235]:
req_announcement = requests.get('https://vk.com/wall-33327655_2225')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="#выборы@stud_mm" />
        <meta property="og:description" content="#выборы@stud_mm" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-

In [236]:
# заведомо бракованная ссылка выглядит так:

req_announcement = requests.get('https://vk.com/wall-33327655_2225')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="#выборы@stud_mm" />
        <meta property="og:description" content="#выборы@stud_mm" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-

In [207]:
# ссылка на коммент выглядит так (как ссылка на родительский пост):

# когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
# когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />

req_announcement = requests.get('https://vk.com/wall-33327655_8800')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="Приостановка сбора заявлений на ПГАС" />
        <meta property="og:description" content="Приостановка сбора заявлений на ПГАС" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa

In [95]:
# вытаскивает весь текст, но так пока не нужно.

from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://vk.com/wall-33327655_2225"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out
# get text
text = soup.get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)
print(text)

#выборы@stud_mm Насыщенная выборная.. | Студенческий совет мехмата МГУ | ВКонтакте
Запись на стене
Студенческий совет мехмата МГУ
19 ноя 2014 в 21:07
#выборы@stud_mmНасыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)— НАГОРНЫХ Яна Валерьевна, группа 108«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и яркая внеучебная жизнь, вот и мне хоче

In [98]:
# КОГДА ДОПИСАЛИ КАНДИДАТОВ В КОММЕНТ (единичный случай, забить)


from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://vk.com/wall-33327655_2225"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

text_by_strings = [text for text in soup.stripped_strings]
print(text_by_strings)

# for line in text_by_strings:
    

['#выборы@stud_mm Насыщенная выборная.. | Студенческий совет мехмата МГУ | ВКонтакте', 'Запись на стене', 'Студенческий совет мехмата МГУ', '19 ноя 2014 в 21:07', '#выборы@stud_mm', 'Насыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.', 'Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».', '1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)', '— НАГОРНЫХ Яна Валерьевна, группа 108', '«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и ярка